# Information about the notebook

This notebook is an example of PyTorch Inference for ongoing Cassava Leaf Disease Classification. 

For the training of the model, I have forked this excellent [notebook](https://www.kaggle.com/sachinprabhu/pytorch-resnet50-snapmix-train-pipeline) with little tweak of label smoothing loss. **Resnet50 + SnapMix + Label Smoothing Loss** combinations are used for training of the models.

Label Smoothing Loss is implemented like this - [Source](https://medium.com/towards-artificial-intelligence/how-to-use-label-smoothing-for-regularization-aa349f7f1dbb)  


    def linear_combination(x, y, epsilon): 
        return epsilon*x + (1-epsilon)*y

    def reduce_loss(loss, reduction='mean'):
        return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss


    class LabelSmoothingCrossEntropy(nn.Module):
        def __init__(self, epsilon:float=0.1, reduction='mean'):
            super().__init__()
            self.epsilon = epsilon
            self.reduction = reduction
    
        def forward(self, preds, target):
            n = preds.size()[-1]
            log_preds = F.log_softmax(preds, dim=-1)
            loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
            nll = F.nll_loss(log_preds, target, reduction=self.reduction)
            return linear_combination(loss/n, nll, self.epsilon)


# Import required libs

In [ ]:
!pip install ../input/timmwhl/timm-0.3.3-py3-none-any.whl

In [ ]:
import random
import os
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import timm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from tqdm import tqdm
import torch.nn.functional as F

import albumentations as A
from albumentations import Compose
from albumentations.pytorch import ToTensorV2
import cv2

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Adding sys path for offline import

In [ ]:
sys.path.append('../input/pytorchimagemodelsmaster/pytorch-image-models-master')

# Required Variables

In [ ]:
DATA_PATH = '../input/cassava-leaf-disease-classification/'
bs = 16
sz = 448
TIMM_MODEL = 'resnet50'

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 1234
seed_everything(SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Cassava Dataset

In [ ]:
class CassavaDataset(Dataset):
    
    def __init__(self, dataframe, root_dir, transforms=None):
        super().__init__()
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.dataframe)
    
    def get_img_bgr_to_rgb(self, path):
        im_bgr = cv2.imread(path)
        im_rgb = im_bgr[:, :, ::-1]
        return im_rgb

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.root_dir,
                                self.dataframe.iloc[idx, 0])
        image = self.get_img_bgr_to_rgb(img_name)
        if self.transforms:
            image = self.transforms(image=image)['image']
        csv_row = self.dataframe.iloc[idx, 1:]
        sample = {
            'image': image, 
            'label': csv_row.label,
        }
        return sample

# Transforms for test data using albumentations

In [ ]:
def test_transforms():
    return Compose([
            A.Resize(sz, sz),
            A.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225], 
                        max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

# Model

In [ ]:
class CassavaNet(nn.Module):
    def __init__(self):
        super().__init__()
        backbone = timm.create_model(TIMM_MODEL, pretrained=False)
        n_features = backbone.fc.in_features
        self.backbone = nn.Sequential(*backbone.children())[:-2]
        self.classifier = nn.Linear(n_features, 5)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward_features(self, x):
        x = self.backbone(x)
        return x

    def forward(self, x):
        feats = self.forward_features(x) #self.backbone(x)
        x = self.pool(feats).view(x.size(0), -1) # avg pool and flattening
        x = self.classifier(x) # Linear classifier
        return x, feats

In [ ]:
model = CassavaNet().to(device)

# Inference

In [ ]:
def predict(model, ckpts, dataloader):    
    predict_list=[]
    with torch.no_grad():
            for _, data in enumerate(dataloader):
                avg_preds = []
                for ckpt in ckpts:
                    model.load_state_dict(ckpt['state_dict'])
                    model.eval()                    
                    images, label = data.values()
                    images =  images.to(device)
                    outputs, _ = model(images)
                    preds = F.softmax(outputs).to('cpu').numpy()
                    avg_preds.append(preds)                
                predict_list.append(np.mean(avg_preds, axis=0))            
            predict_list = np.concatenate(predict_list)
                
    return predict_list.argmax(axis=1)

# Data Loading

In [ ]:
test_df = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

test_dir = '../input/cassava-leaf-disease-classification/test_images/'

test_ds = CassavaDataset(dataframe=test_df,
                         root_dir=test_dir,
                         transforms=test_transforms())

test_dl = DataLoader(test_ds, batch_size=bs, 
                                      shuffle=False, num_workers=8, 
                                      pin_memory=True)

# Model Loading

In [ ]:
ckpts=[]
trained_model_path = "../input/cassavalblsmoothingresnet50"
for path in os.listdir(trained_model_path):
    ckpts.append(torch.load(os.path.join(trained_model_path, path)))


# Prediction

In [ ]:
test_predict_list=predict(model, ckpts, test_dl)

In [ ]:
test_predict_list

# Submission 

In [ ]:
test_df

In [ ]:
test_df['label'] = test_predict_list
test_df[['image_id', 'label']].to_csv('submission.csv', index=False)
test_df.head()

### If you like the notebook, please upvote !!